# Dividing the images into 128 x 128 patches

## 1. Importing Libraries and Set Paths

In [1]:
# === Import Required Libraries ===
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from tqdm import tqdm


## 2. Defining Directories and Settings

### Patch size is fixed to 128x128 for compatibility with most CNN architectures.


In [2]:
# === Define Input and Output Paths ===
toa_dir = r"C:\nrsc_test\toa_tif_outputs"               # Folder with TOA .tif files (BAND2, BAND3, BAND4)
mask_dir = r"C:\nrsc_test\cloud-masks"                  # Folder with corresponding binary cloud masks
output_image_dir = r"C:\nrsc_test\npy_patches\images"   # Save extracted image patches here
output_mask_dir = r"C:\nrsc_test\npy_patches\masks"     # Save corresponding mask patches here
patch_size = 128                                        # Patch size (128x128)


In [3]:
# === Create Output Folders if Not Exist ===
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)


## 3. Extract Scene IDs from TOA Files

Each scene is uniquely identified by its name prefix.
This cell collects all scene IDs by scanning `BAND2` files.
These IDs are used to locate all three bands and the corresponding mask.


In [4]:
# === Extract scene IDs from available BAND2 TOA files ===
def extract_scene_id(filename):
    return filename.replace("_BAND2_toa.tif", "")

scene_ids = [
    extract_scene_id(f)
    for f in os.listdir(toa_dir)
    if f.endswith("_BAND2_toa.tif")
]


## 4. Extract Patches for Each Scene

- Loops over each scene
- Constructs paths to its 3-band TOA images and cloud mask
- Skips the scene if any required file is missing


In [5]:
patch_counter = 0  # To keep count of valid patches saved

# === Loop through each scene ===
for scene_id in tqdm(scene_ids, desc="🔄 Processing scenes"):
    print(f"\n📂 Scene: {scene_id}")

    # Prepare paths for the 3 TOA bands
    band_paths = {
        band: os.path.join(toa_dir, f"{scene_id}_{band}_toa.tif")
        for band in ["BAND2", "BAND3", "BAND4"]
    }
    # Path to corresponding mask
    mask_path = os.path.join(mask_dir, f"cloud_{scene_id}.tif")

    # Skip if any band or mask is missing
    missing = [p for p in list(band_paths.values()) + [mask_path] if not os.path.exists(p)]
    if missing:
        print(f"[SKIP] Missing files for {scene_id}:")
        for m in missing:
            print(f"   ✘ {m}")
        continue


🔄 Processing scenes: 100%|██████████| 1/1 [00:00<00:00, 991.56it/s]


📂 Scene: RAF23MAR2025043024009400045SSANSTUC00GTDC


## 5. Extract and Save Patches (Nested Loop)

###
- Iterates over the image using a sliding window
- For each patch:
  - Reads 3-band TOA image data
  - Checks if the patch is not empty (all zeros)
  - Reads and binarizes the cloud mask
  - Saves both image and mask as `.npy` files

In [6]:
    # Read the shape from BAND2 to determine patch grid
    with rasterio.open(band_paths["BAND2"]) as src:
        h, w = src.height, src.width

    total_patches = (h // patch_size + 1) * (w // patch_size + 1)

    # Extract patches with nested tqdm bar
    with tqdm(total=total_patches, desc=f"🧩 Patches from {scene_id}", leave=False) as pbar:
        for y in range(0, h, patch_size):
            for x in range(0, w, patch_size):
                bands = []
                empty_patch = True

                # Load 3 bands patch-wise
                for band in ["BAND2", "BAND3", "BAND4"]:
                    with rasterio.open(band_paths[band]) as src:
                        window = Window(x, y, patch_size, patch_size)
                        patch = src.read(1, window=window, boundless=True, fill_value=0)
                        bands.append(patch)
                        if np.any(patch > 0):
                            empty_patch = False

                if empty_patch:
                    pbar.update(1)
                    continue  # Skip patches with all zero values

                image_patch = np.stack(bands, axis=0).astype(np.float32)

                # Load corresponding mask patch
                with rasterio.open(mask_path) as src:
                    mask_patch = src.read(1, window=Window(x, y, patch_size, patch_size), boundless=True, fill_value=0)
                    mask_patch = (mask_patch > 0).astype(np.uint8)  # Ensure binary

                # Save patch to disk
                np.save(os.path.join(output_image_dir, f"patch_{patch_counter:06d}_image.npy"), image_patch)
                np.save(os.path.join(output_mask_dir, f"patch_{patch_counter:06d}_mask.npy"), mask_patch)

                patch_counter += 1
                pbar.update(1)


In [7]:
print(f"\n✅ Saved {patch_counter} valid patches.")



✅ Saved 14815 valid patches.
